<a href="https://colab.research.google.com/github/yenee/class/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Import



In [ ]:
!pip install pingouin

In [3]:
import pingouin as pg
import pandas as pd
import io

In [4]:
import statsmodels.formula.api as smf


def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#2.Reading data


In [36]:
from google.colab import files
uploaded = files.upload()

Saving mediandatabird.csv to mediandatabird (2).csv


In [37]:
df = pd.read_csv(io.BytesIO(uploaded['mediandatabird.csv']),encoding='GB18030')
df = df.drop(columns=['id','land','water','num','Pw','Pc'])
df.columns = ["outbreak","FF","MD","FW","FO","WL","UB","WB","BU","BL","長趾濱鷸","紅嘴鷗","小","田鷸","尖尾鴨","太平洋金斑","黑翅長腳鷸","小燕鷗","家燕","黃頭鷺","PHI"]
df
# Dataset is now stored in a Pandas Dataframe

,outbreak,FF,MD,FW,FO,WL,UB,WB,BU,BL,...,紅嘴鷗,小,田鷸,尖尾鴨,太平洋金斑,黑翅長腳鷸,小燕鷗,家燕,黃頭鷺,PHI
0,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,1,0,1,1,0.00
1,0,1,1,0,1,0,1,0,1,0,...,0,0,0,0,0,1,0,0,1,0.00
2,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,1,1,0,1,1,0.20
3,1,0,1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0.00
4,1,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,0,1,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0.00
1069,0,1,1,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0.00
1070,0,1,0,0,1,0,1,0,1,0,...,0,0,1,0,0,1,0,1,1,0.67
1071,0,1,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,1,0,0.00


#3.Regression Model


檢查解釋變數共線性

In [13]:
X = df[["FF","MD","FW","FO","WL","UB","WB","BU","BL","長趾濱鷸","紅嘴鷗","小","田鷸","尖尾鴨","太平洋金斑","黑翅長腳鷸","小燕鷗","家燕","黃頭鷺","PHI"]]
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

   feature       VIF
0       FF  2.380984
1       MD  2.128165
2       FW  3.046372
3       FO  5.883675
4       WL  2.038363
5       UB  2.242118
6       WB  2.319907
7       BU  6.252079
8       BL  1.082843
9     長趾濱鷸  2.013180
10     紅嘴鷗  1.703656
11       小  1.975015
12      田鷸  1.860415
13     尖尾鴨  1.775713
14   太平洋金斑  2.082723
15   黑翅長腳鷸  2.942905
16     小燕鷗  2.073074
17      家燕  5.708494
18     黃頭鷺  6.130247
19     PHI  1.750764


逐步回歸

In [40]:
model = forward_selected(df,'outbreak')
print(model.model.formula)

outbreak ~ PHI + BU + UB + FW + 黑翅長腳鷸 + FO + FF + 長趾濱鷸 + 1


In [42]:
lm = pg.linear_regression(df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']], df['outbreak'],relimp=True)

#4.LMG

In [48]:
lmg = pd.DataFrame(lm[['names','relimp']])
lmg = lmg.drop(lmg.index[0])
lmg.columns = ['var','lmg']
lmg.sort_values(by=['lmg'], ascending=False)

,var,lmg
1,PHI,0.139093
6,FO,0.033282
4,FW,0.032627
2,BU,0.025656
5,黑翅長腳鷸,0.017033
3,UB,0.014469
8,長趾濱鷸,0.004656
7,FF,0.002200


#5.RF

In [14]:
X = df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
y = df['outbreak'] 

初始隨機森林模型

In [16]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
oob_error = 1 - rf0.oob_score_
oob_error

0.23485554520037277

調參數

In [126]:
param_test1 = {'n_estimators':range(100,501,50)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

({'n_estimators': 200}, 0.8330065469431244)

In [127]:
param_test3 = {'min_samples_split':range(80,150,20), 'min_samples_leaf':range(10,60,10)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 200,
                                max_features='sqrt' ,oob_score=True, random_state=10),
                                param_grid = param_test3, scoring='roc_auc',cv=5)
gsearch3.fit(X,y)
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_leaf': 10, 'min_samples_split': 120}, 0.8340668433079588)

In [132]:
param_test4 = {'max_features':range(3,8,1)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                              min_samples_leaf=10 ,oob_score=True, random_state=10),
                              param_grid = param_test4, scoring='roc_auc', cv=5)
gsearch4.fit(X,y)
gsearch4.best_params_, gsearch4.best_score_

({'max_features': 4}, 0.8330230194445611)

最終模型

In [17]:
rf2 = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf2.fit(X,y)
oob_error = 1 - rf2.oob_score_
oob_error

0.2078285181733458

In [24]:
#delete PHI
Test = df[['BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error1 = 1 - rf.oob_score_
IncMSE_x1 = (oob_error1-oob_error)/oob_error*100
IncMSE_x1

15.246636771300428

In [25]:
#delete BU 
Test = df[['PHI','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error2 = 1 - rf.oob_score_
IncMSE_x2 = (oob_error2-oob_error)/oob_error*100
IncMSE_x2

2.690582959641246

In [26]:
#delete UB 
Test = df[['PHI','BU','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error3 = 1 - rf.oob_score_
IncMSE_x3 = (oob_error3-oob_error)/oob_error*100
IncMSE_x3

0.0

In [27]:
#delete FW 
Test = df[['PHI','BU','UB','黑翅長腳鷸','FO','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error4 = 1 - rf.oob_score_
IncMSE_x4 = (oob_error4-oob_error)/oob_error*100
IncMSE_x4

0.4484304932735232

In [30]:
#delete 黑翅長腳鷸 
Test = df[['PHI','BU','UB','FW','FO','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error5 = 1 - rf.oob_score_
IncMSE_x5 = (oob_error5-oob_error)/oob_error*100
IncMSE_x5

-0.8968609865470998

In [31]:
#delete FO 
Test = df[['PHI','BU','UB','FW','黑翅長腳鷸','FF','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error6 = 1 - rf.oob_score_
IncMSE_x6 = (oob_error6-oob_error)/oob_error*100
IncMSE_x6

-0.8968609865470998

In [32]:
#delete FF X = df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
Test = df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','長趾濱鷸']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error7 = 1 - rf.oob_score_
IncMSE_x7 = (oob_error7-oob_error)/oob_error*100
IncMSE_x7

-0.8968609865470998

In [33]:
#delete 長趾濱鷸 
Test = df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF']]
rf = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf.fit(Test,y)
oob_error8 = 1 - rf.oob_score_
IncMSE_x8 = (oob_error8-oob_error)/oob_error*100
IncMSE_x8

0.0

In [51]:
grades = {
    "var": ['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸'],
    "oob_error": [oob_error1, oob_error2, oob_error3, oob_error4, oob_error5, oob_error6, oob_error7, oob_error8],
    "%IncMSE": [IncMSE_x1, IncMSE_x2, IncMSE_x3, IncMSE_x4, IncMSE_x5, IncMSE_x6, IncMSE_x7, IncMSE_x8]
}
 
rf_inc = pd.DataFrame(grades)
rf_inc
rf_inc.sort_values(by=['%IncMSE'], ascending=False)

In [52]:
rf_inc1 = rf_inc[['var','%IncMSE']]
rf_inc1.sort_values(by=['%IncMSE'], ascending=False)

,var,%IncMSE
0,PHI,15.246637
1,BU,2.690583
3,FW,0.448430
2,UB,0.000000
7,長趾濱鷸,0.000000
4,黑翅長腳鷸,-0.896861
5,FO,-0.896861
6,FF,-0.896861
